# OME-Zarr ROI processing
This notebook shows how to load a whole OME-Zarr image, apply some processing to it and store the results as a new label image into the OME-Zarr

### Load whole OME-Zarr image, process it, store resulting labels back into OME-Zarr 

In [ ]:
import zarr
import dask.array as da
import numpy as np
from skimage.measure import label
from skimage.filters import threshold_otsu
from skimage.morphology import closing, square, remove_small_holes, remove_small_objects
import napari

In [ ]:
# TODO: Change to download the zenodo example data, run it on those
zarr_url = "/Users/joel/Desktop/20230510_Fractal_web_test/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr/B/03/0"
level = 0
channel_index = 0

img = da.from_zarr(f"{zarr_url}/{level}")[channel_index]

In [ ]:
img

In [ ]:
# Convert it to a numpy array, do arbitrary processing with the image
# Depending on the processing you want to do, it may also work directly in dask
min_size=256
img = np.array(img)
otsu_threshold = threshold_otsu(img)
img_thr = img > otsu_threshold
img_thr = remove_small_holes(img_thr)
img_thr_cleaned = remove_small_objects(img_thr, min_size=min_size)

In [ ]:
label_image = label(img_thr_cleaned)
label_image.shape

In [ ]:
# Have a look at the label image in napari
viewer = napari.Viewer()
viewer.add_image(img)
label_layer = viewer.add_labels(label_image)

In [ ]:
# Optionally modify the label layer manually in napari, then get that modified label layer
label_image = label_layer.data

### Save the resulting label image back to the OME-Zarr file

In [ ]:
from ome_zarr.writer import write_labels

In [ ]:
new_label_name = "manual_thresholding_1"
with zarr.open(zarr_url, mode='rw+') as zarr_img:
    write_labels(
        label_image,
        zarr_img,
        name = new_label_name,
        axes = "zyx",
        storage_options = {"dimension_separator": '/'}
    )

    # Hacky way of ensuring we have the correct metadata, because the writer 
    # doesn't get the metadata right yet.
    # This assumes the output labels have the same shape as the loaded image
    coordinate_transforms = zarr_img.attrs["multiscales"][0]['datasets']
    axes = zarr_img.attrs["multiscales"][0]["axes"]
    labels_zarr = zarr_img[f"labels/{new_label_name}"]
    multiscales = labels_zarr.attrs['multiscales']
    multiscales[0]['datasets'] = coordinate_transforms
    # Skip the channels axis, because this image contains no channels
    multiscales[0]["axes"] = axes[1:]
    labels_zarr.attrs['multiscales'] = multiscales
